Best Response Sequences in the Iterated Prisoner's Dilemma
------------

This Chapter explores best response strategies, to a collection of opponents, in the form of sequences. The APL project is used to simulate matches between these opponents and the "sequence" players. Morespecifically, the class `Cycler` is used to simulate the sequences.

In [1]:
import axelrod as axl

**Example of $S = \{D, D, D, C, C, C, D, D, C, C\}$ against Cooperator**

In [2]:
players = [axl.Cycler('DDDCCCDDCC'), axl.Cooperator()]

In [3]:
match = axl.Match(players, turns=10)
match.play()

[(D, C),
 (D, C),
 (D, C),
 (C, C),
 (C, C),
 (C, C),
 (D, C),
 (D, C),
 (C, C),
 (C, C)]

In [4]:
match.final_score_per_turn()

(4.0, 1.5)

**Example of$S = \{D, D, D, C, C, C, D, D, C, C\}$ against Tit For Tat**

In [5]:
players = [axl.Cycler('DDDCCCDDCC'), axl.TitForTat()]

In [6]:
match = axl.Match(players, turns=10)
match.play()

[(D, C),
 (D, D),
 (D, D),
 (C, D),
 (C, C),
 (C, C),
 (D, C),
 (D, D),
 (C, D),
 (C, C)]

In [7]:
match.final_score_per_turn()

(2.2, 2.2)

**Example against Random**

In [8]:
players = [axl.Random(), axl.Cycler('DDDCCCDDCC')]

In [9]:
axl.seed(1)
match = axl.Match(players, turns=10)
actions = match.play()

In [10]:
actions

[(C, D),
 (D, D),
 (D, D),
 (C, C),
 (C, C),
 (C, C),
 (D, D),
 (D, D),
 (C, C),
 (C, C)]

In [11]:
match.final_score_per_turn()

(1.9, 2.4)

In [12]:
axl.seed(2)
match = axl.Match(players, turns=10)
actions = match.play()

In [13]:
match.final_score_per_turn()

(2.6, 1.6)

This work uses random seeds to capture different behaviours of stochastic strategies. Random seeding also makes it possible to reproduce the behaviour of stochastic strategies.

In [14]:
players = [axl.Cycler('DDDCCCDDCC'), axl.Random()]
for seed in range(5):
    axl.seed(seed)
    match = axl.Match(players, turns=10)
    actions = match.play()
    print(actions, match.final_score_per_turn())
    print("================================================================================")


[(D, D), (D, D), (D, C), (C, C), (C, D), (C, C), (D, D), (D, C), (C, C), (C, D)] (2.2, 2.2)
[(D, C), (D, D), (D, D), (C, C), (C, C), (C, C), (D, D), (D, D), (C, C), (C, C)] (2.4, 1.9)
[(D, D), (D, D), (D, C), (C, C), (C, D), (C, D), (D, D), (D, C), (C, D), (C, D)] (1.6, 2.6)
[(D, C), (D, D), (D, C), (C, D), (C, D), (C, C), (D, C), (D, D), (C, C), (C, C)] (2.6, 2.1)
[(D, C), (D, C), (D, C), (C, C), (C, C), (C, C), (D, D), (D, D), (C, D), (C, C)] (2.9, 1.9)


`sequence_sensei` is a package which has been developed to perform a genetic algorithm on sequences.

In [24]:
import sequence_sensei as ss
import numpy as np

Generating the starting population is done using the following code:

In [25]:
initial_population = ss.get_initial_population(half_size_of_population=5, sequence_length=8)

In [28]:
np.matrix(initial_population)

matrix([[1, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0]])

In [27]:
np.matrix([[axl.Action(gene) for gene in member] for member in initial_population])

matrix([[D, C, C, C, C, C, C, C],
        [C, D, D, D, D, D, D, D],
        [D, D, C, C, C, C, C, C],
        [C, C, D, D, D, D, D, D],
        [D, D, D, D, C, C, C, C],
        [C, C, C, C, D, D, D, D],
        [D, D, D, D, D, D, C, C],
        [C, C, C, C, C, C, D, D],
        [D, D, D, D, D, D, D, D],
        [C, C, C, C, C, C, C, C]], dtype=object)

`sequence_sensei` includes functions for the crossover and mutation properties of a genetic algorithm.

In [18]:
import random
import sequence_sensei as ss

In [19]:
turns = 10

cooperator = [1 for _ in range(turns)]
alternator = [i % 2 for i in range(turns)]

In [20]:
random.seed(1)
new_member = ss.crossover(cooperator, alternator)

In [21]:
new_member

[1, 1, 0, 1, 0, 1, 0, 1, 0, 1]

In [22]:
random.seed(1)
[ss.mutation(gene, mutation_probability=0.05) for gene in new_member] 

[1, 1, 0, 1, 0, 1, 0, 1, 0, 0]